# SQL query from table names - Continued

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [3]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [4]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [5]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [6]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [11]:
#new
context_user = context.copy()
print(return_CCRMSQL("""List the names and emails of customers who placed an order in the last 90 days.""", context_user))

```sql
SELECT c.name, c.email
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_date >= DATE('now', '-90 days');
```


In [12]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("List the names and emails of customers who placed an order in the last 90 days.", old_context_user))

This is your SQL:
```sql
SELECT name, email
FROM customers
WHERE order_date >= DATE_SUB(CURDATE(), INTERVAL 90 DAY);
```

This SQL query selects the names and emails of customers from the "customers" table who placed an order in the last 90 days by comparing the order date with the current date minus 90 days.


In [13]:
#new
print(return_CCRMSQL("Which customers spent more than $500 total?", context_user))

```sql
SELECT customer_id, SUM(amount) AS total_spent
FROM orders
GROUP BY customer_id
HAVING total_spent > 500;
```


In [14]:
#old
print(return_CCRMSQL("Which customers spent more than $500 total?", old_context_user))

This is your SQL:
```sql
SELECT e.ID_usr, e.name, SUM(s.salary) AS total_salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
GROUP BY e.ID_usr, e.name
HAVING SUM(s.salary) > 500;
```

This SQL query retrieves the ID and name of employees who have spent more than $500 in total. It joins the "employees" and "salary" tables on the employee ID, calculates the total salary for each employee using the SUM function, groups the results by employee ID and name, and then filters the results to only include those with a total salary greater than $500.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

What I learned:

Schema context matters – explicit DDL or JSON definitions reduce guesswork.

Prompt shots – one-shot already stabilizes format; few-shot adds robustness for multi-table joins.

Hallucination hotspots – column names and date literals; always run or parse SQL before trusting.

Temperature 0 keeps answers deterministic; higher temps increase creativity and risk.

Iterative prompting – fastest fix is to feed the erroneous SQL straight back (“this column doesn’t exist, revise”).